### Demo for 3D point cloud registration (with equal number of points)

In [120]:
import numpy as np
from scipy.spatial.transform import Rotation as R

In [121]:
# sample 10 random points in 3D
x = np.random.randn(10,3)

# establish some rotation
rot = R.from_euler('z', 10, degrees=True)
# establish some translation 
t = np.random.randn(1,3)*0.1
# ... and some scaling
scale = 2.0

# get second point cloud as sRx
y = scale*rot.apply(x) + t

In [107]:
# def get_scale(y,x):
#     return np.sqrt(np.sum(np.linalg.norm(yp,axis=1)**2.0)/np.sum(np.linalg.norm(xp,axis=1)**2.0))

In [122]:
# H = np.zeros((3,3))
# for i in range(xp.shape[0]):
#     H += np.outer(xp[i],yp[i])

In [133]:
def rigid_transform_3D(A, B):
    """
    A and B have to be numpy arrays of the form (3,N), i.e.,
    N points in 3D.
    """
    
    assert A.shape == B.shape

    # find mean column wise
    centroid_A = np.mean(A, axis=1)
    centroid_B = np.mean(B, axis=1)

    # ensure centroids are 3x1
    centroid_A = centroid_A.reshape(-1, 1)
    centroid_B = centroid_B.reshape(-1, 1)

    # subtract mean (as on slide 6)
    Am = A - centroid_A
    Bm = B - centroid_B

    # compute matrix H (as on slide 12), i.e., the sum over the outer products
    H = Am @ np.transpose(Bm)

    # find rotation via SVD and compute R as V*H' (as on slide 12)
    U, S, Vt = np.linalg.svd(H)
    R = Vt.T @ U.T

    # next, as we know rotation, we compute scaling
    scale = np.sqrt(np.sum(np.linalg.norm(Bm.transpose(),axis=1)**2.0)/np.sum(np.linalg.norm(Am.transpose(),axis=1)**2.0))
    
    # special reflection case
    if np.linalg.det(R) < 0:
        print("det(R) < R, reflection detected!, correcting for it ...")
        Vt[2,:] *= -1
        R = Vt.T @ U.T

    t = -scale * R @ centroid_A + centroid_B

    return R, t, scale

In [134]:
est_rot,est_t,est_scale = rigid_transform_3D(x.transpose(),y.transpose())

In [149]:
print('Fro. norm diff. in rotation : {:.20f}'.format(np.linalg.norm(est_rot - rot.as_matrix(),'fro')))
print('Abs. diff. in scaling:        {:.20f}'.format(np.abs(scale - est_scale)))
print('Norm. diff in transation:     {:.20f}'.format(np.linalg.norm(t.ravel()-est_t.ravel())))

Fro. norm diff. in rotation : 0.00000000000000050916
Abs. diff. in scaling:        0.00000000000000000000
Norm. diff in transation:     0.00000000000000062222
